<img src="Tutorial-DirectStitch-figs/Chameleon-FullColor-lg_4CCULFA.jpg" width="90%">



# Tutorial Using BYOC OpenFlow Networks on Chameleon

- **Estimated time**: ?? minutes
- **Requirements**:

     - This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon (see: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html)
     - Understanding of Chameleon networks and isolated VLANs. (see: https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html)
     - Basic understanding of the CLI (see https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html)

The Chameleon Bring-Your-Own-Controller (BYOC) capability enables user controlled OpenFlow experiments. BYOC users are able to access isolated OpenFlow switches connected to their own bare metal Chameleon nodes.

During this tutorial we will:

This tutorial will guide you through setting up bring-your-own-controller (BYOC) OpenFlow networks.


- Step 1. Create an OpenFlow controller on Chameleon
- Step 2. Create an OpenFlow network on Chameleon
- Step 3. Connect Chameleon nodes to the OpenFlow netork.
- Step 4. Observe/modify the OpenFlow controller connected to the network.

This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon and basic use of Chameleon networks and isolated VLANs. 

Additional information:

- Getting started tutorial: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html
- Networks and Isolated VLANs:  https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html
- Setting up the CLI: https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html

# Background: 

Chameleon is composed of two OpenStack sites with deeply programmable compute, storage, and networking infrastructure. This tutorial focuses on advanced networking features including Bring-your-own-controller (BYOC) software defined networking (SDN). 

### Basic Chameleon Networking

Each of the two Chameleon sites is includes 100 Gbps uplink connectivity to shared core network that can be used to connect Chameleon nodes to core services, other Chameleon nodes, and even external facilities including GENI.  By default, each Chameleon node can access the public Internet through the core network. Advanced experiments can allocate tenant controlled SDN networks and isolated layer2 circuits across the core to connect Chameleon nodes and external facilities. 

### Software Defined Networking on Chameleon

Chameleon’s Bring Your Own Controller (BYOC) functionality enables tenants to create isolated network switches managed using OpenFlow controllers provided by the user. This feature is targeted at users wishing to experiment with software-defined networking (SDN) as well as users with experiments that have non-standard networking requirements. 

OpenFlow switches, like traditional switches, forward network traffic between a number of ports used to connect other networks and devices. The primary difference is that OpenFlow switches rely on external software (a “controller”) to dynamically manage the rules (or “flows”) that determine how and where the traffic is forwarded. In addition, OpenFlow enables a much larger set of possible rules which can be imposed on the traffic.

The basic requirements of an OpenFlow network are the switch and the controller. The switch is configured with the IP address and port of a controller (software) that manages the switch’s rules. When a packet arrives at the switch, the packet is tested against the rules that are known by the switch to determine what action(s) to take. Typically, if there are no rules that apply to a packet, the packet is sent to the controller which replies with a set of rules for that type of packet. The new rules are cached in the switch and applied to subsequent packets until the rules expire or are explicitly removed.

Chameleon provides BYOC networking in order to enable tenants to allocate OpenFlow switches controlled by their own OpenFlow controller(s). This capability is limited to the phase 2 hardware additions that include the Corsa DP2000 series OpenFlow switches and Skylake compute nodes. The Corsa switches are key to enabling the BYOC functionality. These switches provide a native abstraction for the creation of mutually isolated forwarding contexts which can be thought of as independent OpenFlow switches. Each isolated forwarding context can be configured to use its own OpenFlow controller. The Chameleon BYOC functionality enables tenants to specify the IP and port of an arbitrary OpenFlow controller when they create private networks.

### Tutorial:

This Jupyter tutorial will help you setup the following experimental infrastructure. The experiment includes an OpenFlow controller hosted on Chameleon, an OpenFlow swtich, and two nodes connected to the switch.

<img src="Tutorial-DirectStitch-figs/BYOC-Tutorial.png" width="80%">


#### Setup Environment:

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=pruth-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/${SSH_KEY_NAME}

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
export CONTROLLER_RESERVATION_ID='c3f03f7d-b6b3-4888-83c1-afd98b486086'
export NODE_RESERVATION_ID='c3f03f7d-b6b3-4888-83c1-afd98b486086'


#### Create an OpenFlow Controller.  

In general, you can use any OpenFlow 1.3 controller located anywhere accessible on the Internet.  For this tutorial, you can use a Chameleon complex appliance to automatically create a simple Ryu controller hosted on a Chameleon node. The controller can be used as a base for building more complex controllers.

In [ ]:
# Set the name of the orcestration stack. We suggest embedding your user
# name or some other identifiable string to make it easier to find
# you nodes.  This is especially important if you are in a formal 
# tutorial with many other participants.
CONTROLLER_STACK_NAME=${USERNAME}"_controller_stack"

# Set the controller node name. See above about using 
# identifiable names.
CONTROLLER_NODE_NAME=${USERNAME}"_controller"

# Set the network that the controller uses to communicate with the swtich.
# This networks must be accessible on the Internet and will not be the network
# that the controller is managing. We suggest using "sharednet1"
CONTROLLER_NETWORK="sharednet1"

#Configure the OpenFlow port to be used by the 
CONTROLLER_OPENFLOW_PORT=6653

In [ ]:
echo Creating controller. This will take several minutes!
openstack stack create --max-width 80 \
   --template "https://www.chameleoncloud.org/appliances/api/appliances/54/template" \
   --parameter "key_name=${SSH_KEY_NAME}" \
   --parameter "reservation_id=${CONTROLLER_RESERVATION_ID}" \
   --parameter "ryu_port=${CONTROLLER_OPENFLOW_PORT}" \
   --parameter "network_name=${CONTROLLER_NETWORK}" \
   --parameter "controller_name=${CONTROLLER_NODE_NAME}" \
   --wait \
   ${CONTROLLER_STACK_NAME}
   
echo Controller creation complete! 
echo ${CONTROLLER_NODE_NAME} status `openstack server show  --format value -c status ${CONTROLLER_NODE_NAME}`

In [ ]:
CONTROLLER_PUBLIC_IP=`openstack server show  --format value -c addresses ${CONTROLLER_NODE_NAME} | cut -d " " -f 2`
echo The controller public IP is $CONTROLLER_PUBLIC_IP

echo Please wait a few more minutes until the controller is completely configured and ready for logins.

#### View the controller log file

The controller node will be automatically configured with the simple Ryu controller service ready for an OpenFlow switch to attach to it. It will be useful to view the Ryu log file within the controller at various steps in this tutorial, as well as when modifying the controller for your experiment(s).   

You can view the tail of the log file in thie notbook with the following cell.  A jupyter notebook can only run one cell at a time and cannot continously watch the log as you proceed with the tutorial. You can rerun this cell at anytime but may wish to open a terminal window on you local machine (or within this Jupyter container) and continuously watch the controller log.  

Note that you will likely need to wait several minutes after the controller node is ACTIVE before the controller server is completely configured and ready.

In [ ]:
CONTROLLER_PUBLIC_IP=`openstack server show --format value -c addresses ${CONTROLLER_NODE_NAME} | cut -d " " -f 2`

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CONTROLLER_PUBLIC_IP} \
    tail -n 20 /var/log/ryu/ryu-manager.log 


#### Start the OpenFlow Switch

In [ ]:
# Set the subnet to use on the OpenFlow network
OPENFLOW_NETWORK_SUBNET_CIDR="192.168.100.0/24"

# Set the OpenStack names for the network, subnet, router, and switch. 
# See above about using identifiable names.  
# Note that OPENFLOW_SWITCH_NAME cannot include the '-' character.
OPENFLOW_NETWORK_NAME=${USERNAME}"Network"
OPENFLOW_SUBNET_NAME=${USERNAME}"Subnet"
OPENFLOW_ROUTER_NAME=${USERNAME}"Router"

# Note that OPENFLOW_SWITCH_NAME cannot include the '-' character. 
# This name is used to add additional uplink ports to the same OpenFlow switch.
OPENFLOW_SWITCH_NAME=${USERNAME}"Switch"

In [ ]:
echo Creating network ${OPENFLOW_NETWORK_NAME}
openstack network create --max-width 80 \
                         --provider-network-type vlan \
                         --provider-physical-network physnet1 \
                         --description OFController=${CONTROLLER_PUBLIC_IP}:${CONTROLLER_OPENFLOW_PORT},VSwitchName=${OPENFLOW_SWITCH_NAME} \
                         ${OPENFLOW_NETWORK_NAME}
                         
PRIMARY_UPLINK_VLAN=`openstack network show -c provider:segmentation_id -f value ${OPENFLOW_NETWORK_NAME}`
echo Primary uplink VLAN and port ID: $PRIMARY_UPLINK_VLAN 

#### Observe the controller log file 

You should see that a port was added with the ID of the primary uplink VLAN tag.

In [ ]:
CONTROLLER_PUBLIC_IP=`openstack server show --format value -c addresses ${CONTROLLER_NODE_NAME} | cut -d " " -f 2`

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CONTROLLER_PUBLIC_IP} \
    tail -n 20 /var/log/ryu/ryu-manager.log 


#### Add a subnet and router to the network

In [ ]:
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${OPENFLOW_NETWORK_SUBNET_CIDR} \
                        --dhcp \
                        --network ${OPENFLOW_NETWORK_NAME} \
                        ${OPENFLOW_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${OPENFLOW_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${OPENFLOW_ROUTER_NAME} ${OPENFLOW_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${OPENFLOW_ROUTER_NAME}

echo Network ${OPENFLOW_NETWORK_NAME} is ready for nodes!

#### Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [ ]:
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${NODE_RESERVATION_ID} \
                        --security-group default  \
                        --nic net-id=${OPENFLOW_NETWORK_NAME} \
                        --min 2 \
                        --max 2 \
                        --wait \
                        ${USERNAME}-node

echo Server creation complete! 
echo ${USERNAME}-node-1 is `openstack server show --format value -c status ${USERNAME}-node-1`
echo ${USERNAME}-node-2 is `openstack server show --format value -c status ${USERNAME}-node-2`

### Experiment with you new OpenFlow controller and switch

- Check the controller log to see when the nodes are added to the swtich. 

- Add public IPs, login to the nodes, ping, eachother, etc..  While experimenting with the nodes, continue to  watch the controller log and see all the packet_in calls.

# Please Cleanup Your Resources!!!

Delete your nodes.

If you added more nodes outside of this notebook you will need to delete them as well.  You will not beable to delete the network if there are nodes still attached to it.

In [ ]:
echo Deleting servers ${USERNAME}-node-1 and ${USERNAME}-node-2
openstack server delete --wait ${USERNAME}-node-1 ${USERNAME}-node-2

echo Servers deleted!

Unlink and delete all pieces of the OpenFlow network.

In [ ]:
echo Unlinking router from gateway
openstack router unset --external-gateway ${OPENFLOW_ROUTER_NAME}

echo Unlinking router from subnet
openstack router remove subnet ${OPENFLOW_ROUTER_NAME} ${OPENFLOW_SUBNET_NAME}

echo Deleting router
openstack router delete ${OPENFLOW_ROUTER_NAME}

echo Deleting network
openstack network delete ${OPENFLOW_NETWORK_NAME}

echo All routers, subnets, and networks are deleted!

Delete your OpenFlow controller and stack

In [ ]:
echo Deleting stack ${CONTROLLER_STACK_NAME}
openstack stack delete -y --wait ${CONTROLLER_STACK_NAME}
echo Stack ${CONTROLLER_STACK_NAME} deleted